## Video Load

In [2]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from util import *

In [3]:
"""
contours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                             
contour = contours[0]

epsilon = 0.005 * cv2.arcLength(contour, True)

approx = cv2.approxPolyDP(contour, epsilon, True)

result = cv2.copyTo(frameS04, ROI)

cv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)
"""

'\ncontours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n                             \ncontour = contours[0]\n\nepsilon = 0.005 * cv2.arcLength(contour, True)\n\napprox = cv2.approxPolyDP(contour, epsilon, True)\n\nresult = cv2.copyTo(frameS04, ROI)\n\ncv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)\n'

In [32]:
def load_video(path, ms=25):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    run, frame = capture.read() # 다음 Frame 읽기
    
    prevS04, prevS16, prevS64 = None, None, None
    
    if run: # Frame을 읽은 경우
        frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

        prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            crop = 64

            frame = frame[crop:-crop,:]

            # 영상 사이즈 기준 1920 x 1080

            # 영상의 사이즈 조정 (x1/16 또는 x1/32)
            frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
                            
            hsv = cv2.cvtColor(frameS04, cv2.COLOR_BGR2HSV)

            h, s, v = cv2.split(hsv)

            cv2.imshow("S", s) 

            #h = cv2.GaussianBlur(h, (17, 17), sigmaX=0, sigmaY=0)

            # H 채널에 대한 히스토그램 구하기
            hist_h = cv2.calcHist([h], [0], None, [256], [0, 256])
            
            # 히스토그램 노이즈 제거
            hist_h = cv2.GaussianBlur(hist_h, (1, 3), sigmaX=0, sigmaY=0)

            # 히스토그램 평균, 분산, 표준 편차
            hist_h /= hist_h.sum()

            mean, variance_l, variance_r = 0, 0, 0
            
            for i in range(len(hist_h)):
                mean += hist_h[i] * i

            for i in range(len(hist_h)):
                variance = (hist_h[i] * (i - mean) ** 2) * 2

                if (i <= mean):
                    variance_l += variance
                else:
                    variance_r += variance

            SD_l = math.sqrt(variance_l)
            SD_r = math.sqrt(variance_r)

            # 히스토그램을 이미지로 표시
            hist_img = get_histogram_image(hist_h)

            l, r = int(max(mean - SD_l * 1.35, 0)), int(min(mean + SD_r * 0.4, 255))

            draw_line_on_histogram_image(hist_img, l, (0, 0, 255))
            draw_line_on_histogram_image(hist_img, r, (0, 0, 255))
            draw_line_on_histogram_image(hist_img, int(mean), (255, 0, 255))

            cv2.imshow("Histogram H", hist_img) 

            extraction = cv2.inRange(h, l, r)

            kernel = np.ones((27, 27))
            field = cv2.erode(extraction, kernel)  
            
            kernel = np.ones((18, 18))
            field = cv2.dilate(field, kernel)  

            kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype=np.uint8)
            field = cv2.dilate(field, kernel, iterations=9)  

            contours, hierarchy = cv2.findContours(field, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                                        
            # 필드 영역 마스크 생성
            field_mask = np.zeros(field.shape, dtype=np.uint8)

            for contour in contours:
                hull = cv2.convexHull(contour)

                cv2.drawContours(field_mask, [hull], -1, 255, -1) 

            # 필드 위의 라인에 대한 처리
            blurred_s = cv2.GaussianBlur(s, (3, 3), sigmaX=0, sigmaY=0)

            sobel_x = cv2.Sobel(blurred_s, -1, 1, 0, delta=128)
            sobel_y = cv2.Sobel(blurred_s, -1, 0, 1, delta=128)

            sobel_x = cv2.copyTo(sobel_x, field_mask) 
            sobel_y = cv2.copyTo(sobel_y, field_mask) 

            cv2.imshow("FieldH", field)
            cv2.imshow("Frame", frameS04)
            cv2.imshow("sobel_x", sobel_x)
            cv2.imshow("sobel_y", sobel_y)
            cv2.waitKey(ms) # Millisecond 단위로 대기

            prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거

In [33]:
# Video가 저장된 경로 입력
PATH = r"video/soccer2.mp4"

# Video 재생 및 반환 (Numpy Array)
load_video(PATH, ms=20)

KeyboardInterrupt: 